# Escrow Using Marlowe Runtime\'s REST API

The escrow contract example is a simple Marlowe contract where a seller offers merchandise for sale and a buyer purchases it; if the buyer is not satisfied with the merchandise, they may dispute the purchase, in which case a mediator may rule for the buyer or seller.

In this demonsration we use Marlowe Runtime\'s REST API, served via `marlowe-web-server`, to run this contract on Cardano\'s `preprod` public testnet. Marlowe contracts may use either addresses or role tokens for authorization: here we use role tokens and we have Marlowe Runtime mint them.

In [Marlowe Playground](https://play.marlowe-finance.io/), the contract looks like this in Blockly format.

![Marlowe contract for escrow](images/escrow-playground.png)

In Marlowe format it appears as
```
When [
  (Case
     (Deposit Role "Seller" Role "Buyer"
        (Token "" "")
        (ConstantParam "Price"))
     (When [
           (Case
              (Choice
                 (ChoiceId "Everything is alright" Role "Buyer") [
                 (Bound 0 0)]) Close)
           ,
           (Case
              (Choice
                 (ChoiceId "Report problem" Role "Buyer") [
                 (Bound 1 1)])
              (Pay Role "Seller"
                 (Account Role "Buyer")
                 (Token "" "")
                 (ConstantParam "Price")
                 (When [
                       (Case
                          (Choice
                             (ChoiceId "Confirm problem" Role "Seller") [
                             (Bound 1 1)]) Close)
                       ,
                       (Case
                          (Choice
                             (ChoiceId "Dispute problem" Role "Seller") [
                             (Bound 0 0)])
                          (When [
                                (Case
                                   (Choice
                                      (ChoiceId "Dismiss claim" Role "Mediator") [
                                      (Bound 0 0)])
                                   (Pay Role "Buyer"
                                      (Party Role "Seller")
                                      (Token "" "")
                                      (ConstantParam "Price") Close))
                                ,
                                (Case
                                   (Choice
                                      (ChoiceId "Confirm problem" Role "Mediator") [
                                      (Bound 1 1)]) Close)]
                                      (TimeParam "Mediation deadline") Close))] 
                                      (TimeParam "Complaint response deadline") Close)))] 
                                      (TimeParam "Complaint deadline") Close))] 
                                      (TimeParam "Payment deadline") Close
```

The flow chart below shows the possible execution paths of the escrow contract. This example demonstrates the shaded path.

![Flow chart for escrow contract, with the "dismiss claim" execution path highlighted](images/04-dismiss-claim.svg)

## Preliminaries

See [Lesson 0. Preliminaries](00-preliminaries.md) for information on setting up one's environment for using this tutorial.

The lesson assumes that the following environment variables have been set.
- `CARDANO_NODE_SOCKET_PATH`: location of Cardano node's socket.
- `CARDANO_TESTNET_MAGIC`: testnet magic number.
- `CARDANO_RT_WEBSERVER_HOST`: IP address of the Marlowe Runtime web server.
- `CARDANO_RT_WEBSERVER_PORT`: Port number for the Marlowe Runtime web server.

It also assumes that the parties have addresses, signing keys, and funds.
- Seller
    - [keys/lender.address](keys/lender.address): Cardano address for the seller
    - [keys/lender.skey](keys/lender.skey): location of signing key file for the seller
- Buyer
    - [keys/borrower.address](keys/borrower.address): Cardano address for the buyer
    - [keys/borrower.skey](keys/borrower.skey): location of signing key file for the buyer
- Mediator
    - [keys/mediator.address](keys/mediator.address): Cardano address for the mediator
    - [keys/mediator.skey](keys/mediator.skey): location of signing key file for the mediator

### Access to Cardano node and Marlowe Runtime

If we're using [demeter.run](https://demeter.run/)'s Cardano Marlowe Runtime extension, then we already have access to Cardano Node and Marlowe Runtime. The followind commands will set the required environment variables to use a local docker deployment on the default ports. It will also set some supplementary environment variables.

In [1]:
if [[ -z "$MARLOWE_RT_WEBSERVER_PORT" ]]
then

  if command -v podman > /dev/null
  then
    DOCKER_CLI=podman
  else
    DOCKER_CLI=docker
  fi
  
  # Only required for `marlowe-cli` and `cardano-cli`.
  export CARDANO_NODE_SOCKET_PATH="$($DOCKER_CLI volume inspect marlowe-compose_shared | jq -r '.[0].Mountpoint')/node.socket"
  export CARDANO_TESTNET_MAGIC=1 # Note that preprod=1 and preview=2. Do not set this variable if using mainnet.

  # Only required for `marlowe`.
  export MARLOWE_RT_WEBSERVER_HOST="127.0.0.1"
  export MARLOWE_RT_WEBSERVER_PORT=3780

fi

# FIXME: This should be inherited from the parent environment.
if [[ -z "$CARDANO_NODE_SOCKET_PATH" ]]
then
  export CARDANO_NODE_SOCKET_PATH=/ipc/node.socket
fi

# FIXME: This should be set in the parent environment.
if [[ -z "$CARDANO_TESTNET_MAGIC" ]]
then
  export CARDANO_TESTNET_MAGIC=$CARDANO_NODE_MAGIC
fi

case "$CARDANO_TESTNET_MAGIC" in
  1)
    export "EXPLORER_URL=https://preprod.cardanoscan.io"
    ;;
  2)
    export "EXPLORER_URL=https://preview.cardanoscan.io"
    ;;
  *)
    # Use `mainnet` as the default.
    export "EXPLORER_URL=https://cardanoscan.io"
    ;;
esac

MARLOWE_RT_WEBSERVER_URL="http://$MARLOWE_RT_WEBSERVER_HOST:$MARLOWE_RT_WEBSERVER_PORT"

echo "CARDANO_NODE_SOCKET_PATH = $CARDANO_NODE_SOCKET_PATH"
echo "CARDANO_TESTNET_MAGIC = $CARDANO_TESTNET_MAGIC"
echo "MARLOWE_RT_WEBSERVER_HOST = $MARLOWE_RT_WEBSERVER_HOST"
echo "MARLOWE_RT_WEBSERVER_PORT = $MARLOWE_RT_WEBSERVER_PORT"
echo "MARLOWE_RT_WEBSERVER_URL = $MARLOWE_RT_WEBSERVER_URL"

CARDANO_NODE_SOCKET_PATH = /home/bbush/.local/share/containers/storage/volumes/marlowe-compose_shared/_data/node.socket
CARDANO_TESTNET_MAGIC = 1
MARLOWE_RT_WEBSERVER_HOST = 127.0.0.1
MARLOWE_RT_WEBSERVER_PORT = 3780
MARLOWE_RT_WEBSERVER_URL = http://127.0.0.1:3780


### Seller address and funds

Check that an address and key has been created for the seller. If not, see "Creating Addresses and Signing Keys" in [Lesson 0. Preliminaries](00-preliminaries.md).

In [2]:
SELLER_SKEY=keys/lender.skey
SELLER_ADDR=$(cat keys/lender.address)
echo "SELLER_ADDR = $SELLER_ADDR"

SELLER_ADDR = addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck


Check that the seller has at least one hundred ada.

In [3]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$SELLER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
b32eab58a670cc0164c9c2c615c81b40dc9229ab8d899db8f232bbede0984107     2        1000000000 lovelace + TxOutDatumNone


One can view the address on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [4]:
echo "$EXPLORER_URL/address/$SELLER_ADDR"

https://preprod.cardanoscan.io/address/addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck


### Buyer address and funds

Check that an address and key has been created for the buyer. If not, see "Creating Addresses and Signing Keys" in [Lesson 0. Preliminaries](00-preliminaries.md).

In [5]:
BUYER_SKEY=keys/borrower.skey
BUYER_ADDR=$(cat keys/borrower.address)
echo "BUYER_ADDR = $BUYER_ADDR"

BUYER_ADDR = addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d


Check that the buyer has at least one hundred ada.

In [6]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BUYER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
b32eab58a670cc0164c9c2c615c81b40dc9229ab8d899db8f232bbede0984107     3        1000000000 lovelace + TxOutDatumNone


One can view the address on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [7]:
echo "$EXPLORER_URL/address/$BORROWER_ADDR"

https://preprod.cardanoscan.io/address/


### Mediator address and funds

Check that an address and key has been created for the mediator. If not, see "Creating Addresses and Signing Keys" in [Lesson 0. Preliminaries](00-preliminaries.md).

In [8]:
MEDIATOR_SKEY=keys/mediator.skey
MEDIATOR_ADDR=$(cat keys/mediator.address)
echo "MEDIATOR_ADDR = $MEDIATOR_ADDR"

MEDIATOR_ADDR = addr_test1vr6tytqs3x8qgewhw89m3xrz58t3tqu2hfsecw0u06lf3hg052wsv


Check that the mediator has at least one hundred ada.

In [9]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$MEDIATOR_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
b32eab58a670cc0164c9c2c615c81b40dc9229ab8d899db8f232bbede0984107     4        1000000000 lovelace + TxOutDatumNone


One can view the address on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [10]:
echo "$EXPLORER_URL/address/$MEDIATOR_ADDR"

https://preprod.cardanoscan.io/address/addr_test1vr6tytqs3x8qgewhw89m3xrz58t3tqu2hfsecw0u06lf3hg052wsv


## Design the contract

The escrow contract can be downloaded from the [Marlowe Playground](https://play.marlowe-finance.io/) as a JSON file, or it can be generated using [Marlowe CLI](https://github.com/input-output-hk/marlowe-cardano/tree/main/marlowe-cli#readme) using the `marlowe-cli template` command.

Set the purchase prices to 75 ada.

In [11]:
ADA=1000000  # 1 ada = 1,000,000 lovelace
PRICE=$((75 * ADA))
echo "PRICE = $PRICE lovelace"

PRICE = 75000000 lovelace


On the Cardano blockchain, the protocol parameters require that each UTxO contain at least some ada. Here we will start the contract with 2 ada.

In [12]:
MIN_LOVELACE="$((2 * ADA))"
echo "MIN_LOVELACE = $MIN_LOVELACE lovelace"

MIN_LOVELACE = 2000000 lovelace


Later in the example we will need some constants for converting times.

In [13]:
SECOND=1000 # 1 second = 1000 milliseconds
MINUTE=$((60 * SECOND)) # 1 minute = 60 seconds
HOUR=$((60 * MINUTE)) # 1 hour = 60 minutes

### *Alternative 1:* Use Marlowe Playground to design the contract

If you want to create the contract in Marlowe Playground, do the following:

1. Visit https://play.marlowe-finance.io/ in a web browser.
2. Select "Open an Example".
3. Select "Marlowe" or "Blockly" under "Escrow".
4. Select "Send to Simulator".
5. Set the "Payment deadline" to one hour into the future.
6. Set the "Complaint deadline" to two hours into the future
7. Set the "Complaint response deadline" to three hours into the future.
8. Set the "Mediation deadline" to four hours into the future.
9. Set the "Price" to 75 ada.
10. Select "Download as JSON", set the file name to "escrow-contract.json", and store the file in this folder, namely [marlowe-starter-kit/04-escrow-rest/](.).

![Setting parameters for the escrow bond contract in Marlowe Playground](images/escrow-simulation.png)

### *Alternative 2:* Use Marlowe CLI to generate the contract

Below we generate the contract using Marlowe CLI.

First find the current time, measured in [POSIX milliseconds](https://en.wikipedia.org/wiki/Unix_time).

In [14]:
NOW="$(($(date -u +%s) * SECOND))"
echo "NOW = $NOW POSIX milliseconds = $(date -d @$((NOW / SECOND)))"

NOW = 1679000784000 POSIX milliseconds = Thu Mar 16 03:06:24 PM MDT 2023


The contract has four deadlines. For convenience in this example, set the deadlines to the near future.

In [15]:
PAYMENT_DEADLINE=$((NOW+1*HOUR))    # The payment deadline, one hour from now.
COMPLAINT_DEADLINE=$((NOW+2*HOUR))  # The complaint deadline, two hours from now.
DISPUTE_DEADLINE=$((NOW+3*HOUR))    # The dispute deadline, three hours from now.
MEDIATION_DEADLINE=$((NOW+4*HOUR))  # The mediation deadline, four hours from now.

echo "PAYMENT_DEADLINE = $PAYMENT_DEADLINE POSIX milliseconds = $(date -d @$((PAYMENT_DEADLINE / SECOND)))"
echo "COMPLAINT_DEADLINE = $COMPLAINT_DEADLINE POSIX milliseconds = $(date -d @$((COMPLAINT_DEADLINE / SECOND)))"
echo "DISPUTE_DEADLINE = $DISPUTE_DEADLINE POSIX milliseconds = $(date -d @$((DISPUTE_DEADLINE / SECOND)))"
echo "MEDIATION_DEADLINE = $MEDIATION_DEADLINE POSIX milliseconds = $(date -d @$((MEDIATION_DEADLINE / SECOND)))"

PAYMENT_DEADLINE = 1679004384000 POSIX milliseconds = Thu Mar 16 04:06:24 PM MDT 2023
COMPLAINT_DEADLINE = 1679007984000 POSIX milliseconds = Thu Mar 16 05:06:24 PM MDT 2023
DISPUTE_DEADLINE = 1679011584000 POSIX milliseconds = Thu Mar 16 06:06:24 PM MDT 2023
MEDIATION_DEADLINE = 1679015184000 POSIX milliseconds = Thu Mar 16 07:06:24 PM MDT 2023


Now create the JSON file for the contract, `zcb-contract.json`.

In [16]:
marlowe-cli template escrow \
  --minimum-ada "$MIN_LOVELACE" \
  --price "$PRICE" \
  --seller Seller \
  --buyer Buyer \
  --mediator Mediator \
  --payment-deadline "$PAYMENT_DEADLINE" \
  --complaint-deadline "$COMPLAINT_DEADLINE" \
  --dispute-deadline "$DISPUTE_DEADLINE" \
  --mediation-deadline "$MEDIATION_DEADLINE" \
  --out-contract-file escrow-contract.json \
  --out-state-file /dev/null

The various command-line options are described by the help system.

In [17]:
marlowe-cli template escrow --help

Usage: marlowe-cli template escrow --minimum-ada INTEGER --price INTEGER
                                   --seller PARTY --buyer PARTY --mediator PARTY
                                   --payment-deadline TIMEOUT
                                   --complaint-deadline TIMEOUT
                                   --dispute-deadline TIMEOUT
                                   --mediation-deadline TIMEOUT

  Create an escrow contract.

Available options:
  --minimum-ada INTEGER    Lovelace in the initial state.
  --price INTEGER          The price of the sale, in lovelace.
  --seller PARTY           The seller.
  --buyer PARTY            The buyer.
  --mediator PARTY         The mediator.
  --payment-deadline TIMEOUT
                           The deadline for the buyer to pay. POSIX milliseconds
                           or duration: `INTEGER[s|m|d|w|h]`.
  --complaint-deadline TIMEOUT
                           The deadline for the buyer to complain. POSIX
                           mi

## Examine the contract

View the contract file as YAML.

In [18]:
json2yaml escrow-contract.json

timeout: 1679004384000
timeout_continuation: close
when:
- case:
    deposits: 75000000
    into_account:
      role_token: Seller
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      role_token: Buyer
  then:
    timeout: 1679007984000
    timeout_continuation: close
    when:
    - case:
        choose_between:
        - from: 0
          to: 0
        for_choice:
          choice_name: Everything is alright
          choice_owner:
            role_token: Buyer
      then: close
    - case:
        choose_between:
        - from: 1
          to: 1
        for_choice:
          choice_name: Report problem
          choice_owner:
            role_token: Buyer
      then:
        from_account:
          role_token: Seller
        pay: 75000000
        then:
          timeout: 1679011584000
          timeout_continuation: close
          when:
          - case:
              choose_between:
              - from: 1
                to: 1
              for_choice:
 

### \[Optional\] Check the safety of the contract

If we were running the contract on the Cardano `mainnet`, then we\'d want to check its safety before creating it, so that there is no chance that we might lose funds.

Here are the steps for checking the safety of a contract:

1. Understand the [Marlowe Language](https://marlowe-finance.io/).
2. Understand Cardano\'s [Extended UTxO Model](https://docs.cardano.org/learn/eutxo-explainer).
3. Read and understand the [Marlowe Best Practices Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/best-practices.md).
4. Read and understand the [Marlowe Security Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/security.md).
5. Use [Marlowe Playground](https://play.marlowe-finance.io/) to flag warnings, perform static analysis, and simulate the contract.
6. Use [Marlowe CLI\'s](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe-cli/ReadMe.md) `marlowe-cli run analyze` tool to study whether the contract can run on a Cardano network.
7. Run *all execution paths* of the contract on a [Cardano testnet](https://docs.cardano.org/cardano-testnet/overview).

See [Lesson 1](01-runtime-cli.ipynb) for an example of performing step 6.

## Transaction 1: Mediator Creates Escrow Contract with Initial ADA

A `HTTP` `POST` request to Marlowe Runtime\'s `/contracts` endpoint will build the creation transaction for a Marlowe contract. We provide it the JSON file containing the contract and tell it the `MIN_LOVELACE` value that we previously chose. Anyone could create the contract, but in this example the lender will be doing so, so we provide their address to fund the transaction and to receive the change from it.

First we create the JSON body of the request to build the creation transaction.

In [19]:
yaml2json << EOI > request-1.json
version: v1
contract: $(cat escrow-contract.json)
roles:
  Seller: $SELLER_ADDR
  Buyer: $BUYER_ADDR
  Mediator: $MEDIATOR_ADDR
minUTxODeposit: $MIN_LOVELACE
metadata: {}
tags: {}
EOI
cat request-1.json

{"contract":{"timeout":1679004384000,"timeout_continuation":"close","when":[{"case":{"deposits":75000000,"into_account":{"role_token":"Seller"},"of_token":{"currency_symbol":"","token_name":""},"party":{"role_token":"Buyer"}},"then":{"timeout":1679007984000,"timeout_continuation":"close","when":[{"case":{"choose_between":[{"from":0,"to":0}],"for_choice":{"choice_name":"Everything is alright","choice_owner":{"role_token":"Buyer"}}},"then":"close"},{"case":{"choose_between":[{"from":1,"to":1}],"for_choice":{"choice_name":"Report problem","choice_owner":{"role_token":"Buyer"}}},"then":{"from_account":{"role_token":"Seller"},"pay":75000000,"then":{"timeout":1679011584000,"timeout_continuation":"close","when":[{"case":{"choose_between":[{"from":1,"to":1}],"for_choice":{"choice_name":"Confirm problem","choice_owner":{"role_token":"Seller"}}},"then":"close"},{"case":{"choose_between":[{"from":0,"to":0}],"for_choice":{"choice_name":"Dispute problem","choice_owner":{"role_token":"Seller"}}},"th

Next we post the request and view the response.

In [20]:
curl "$MARLOWE_RT_WEBSERVER_URL/contracts" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $MEDIATOR_ADDR" \
  -d @request-1.json \
  -o response-1.json \
  -sS
json2yaml response-1.json

links:
  contract: contracts/3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20%231
resource:
  contractId: 3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20#1
  txBody:
    cborHex: 86a80081825820b32eab58a670cc0164c9c2c615c81b40dc9229ab8d899db8f232bbede0984107040d81825820b32eab58a670cc0164c9c2c615c81b40dc9229ab8d899db8f232bbede0984107040185a200581d60f4b22c10898e0465d771cbb89862a1d715838aba619c39fc7ebe98dd011a3b45a132a300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a001e8480028200582016d16bfee2695ec0e03c08ec1ca21e1493640130a0cbfcc9a15f2ff181f25a49a200581d604959d439bd61c63c94a102be0defd6a54b36071d6789f05b7b0224a101821a000fb7caa1581cbf35f1fb673d65bd7fc65e90e52213e7c70ceadfab633e9765be060ba145427579657201a200581d60f4b22c10898e0465d771cbb89862a1d715838aba619c39fc7ebe98dd01821a000fea4ca1581cbf35f1fb673d65bd7fc65e90e52213e7c70ceadfab633e9765be060ba1484d65646961746f7201a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc5901821a000f

The identifier for the contract is embedded in the response.

In [21]:
CONTRACT_ID="$(jq -r '.resource.contractId' response-1.json)"
echo "CONTRACT_ID = $CONTRACT_ID"

CONTRACT_ID = 3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20#1


The CBOR serialization (in text-envelope format) is also embedded in the response.

In [22]:
jq '.resource.txBody' response-1.json > tx-1.unsigned

There are many ways to sign and submit Cardano transactions:
- `cardano-cli` at the command line
- `cardano-wallet` at the command line or as a REST service
- `cardano-hw-cli` for a hardware wallet at the command line
- a Babbage-compatible CIP-30 wallet in a web browser
- `marlowe-cli` at the command line

For convenience, here we use `marlowe-cli transaction submit`. One may have to wait a minute or so for the transactions to be confirmed on the blockchain.

In [23]:
TX_1=$(
marlowe-cli transaction submit \
  --tx-body-file tx-1.unsigned \
  --required-signer "$MEDIATOR_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_1 = $TX_1"

TX_1 = 3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20


One can view the transaction on a Cardano explorer and see that the contract has been created and the parties have received their role tokens. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [24]:
echo "$EXPLORER_URL/transaction/$TX_1?tab=utxo"

https://preprod.cardanoscan.io/transaction/3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20?tab=utxo


In particular, we see that the Marlowe contract holds the 2 ada that was set as `MINIMUM_LOVELACE`.

In [25]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$CONTRACT_ID"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20     1        2000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "16d16bfee2695ec0e03c08ec1ca21e1493640130a0cbfcc9a15f2ff181f25a49"


One can see that the seller, buyer, and mediator have received their role tokens. Note that `4c656e646572 = Seller`, `4275796572 = Buyer`, and `4d65646961746f72 = Mediator` in hexadecimal notation.

In [26]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$SELLER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20     4        1034400 lovelace + 1 bf35f1fb673d65bd7fc65e90e52213e7c70ceadfab633e9765be060b.53656c6c6572 + TxOutDatumNone
b32eab58a670cc0164c9c2c615c81b40dc9229ab8d899db8f232bbede0984107     2        1000000000 lovelace + TxOutDatumNone


In [27]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BUYER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20     2        1030090 lovelace + 1 bf35f1fb673d65bd7fc65e90e52213e7c70ceadfab633e9765be060b.4275796572 + TxOutDatumNone
b32eab58a670cc0164c9c2c615c81b40dc9229ab8d899db8f232bbede0984107     3        1000000000 lovelace + TxOutDatumNone


In [28]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$MEDIATOR_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20     0        994418994 lovelace + TxOutDatumNone
3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20     3        1043020 lovelace + 1 bf35f1fb673d65bd7fc65e90e52213e7c70ceadfab633e9765be060b.4d65646961746f72 + TxOutDatumNone


## View the details of the contract on the blockchain

Marlowe Runtime\'s `HTTP` `GET` endpoint `/contracts/{contractId}` can fetch a contract from the blockchain and return information about it.

In [29]:
CONTRACT_URL="$MARLOWE_RT_WEBSERVER_URL/$(jq -r '.links.contract' response-1.json)"
echo "CONTRACT_URL = $CONTRACT_URL"

CONTRACT_URL = http://127.0.0.1:3780/contracts/3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20%231


In [30]:
curl -sS "$CONTRACT_URL" | json2yaml

links:
  transactions: contracts/3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20%231/transactions
resource:
  block:
    blockHeaderHash: ac573deefb3e0762732f5b25d0d1c676d1d0b566c03b447223c882df2c55ba06
    blockNo: 732030
    slotNo: 23317627
  continuations: null
  contractId: 3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20#1
  currentContract:
    timeout: 1679004384000
    timeout_continuation: close
    when:
    - case:
        deposits: 75000000
        into_account:
          role_token: Seller
        of_token:
          currency_symbol: ''
          token_name: ''
        party:
          role_token: Buyer
      then:
        timeout: 1679007984000
        timeout_continuation: close
        when:
        - case:
            choose_between:
            - from: 0
              to: 0
            for_choice:
              choice_name: Everything is alright
              choice_owner:
                role_token: Buyer
          then: close
      

## Transaction 2: Buyer Deposits Funds into Seller’s Account

The buyer deposits their 75 ada into the contract using Marlowe Runtime\'s `HTTP` `POST` `/contract/{contractId}/transactions` endpoint. The buyer is providing the funding for and receiving the change from this transaction, so we provide their address.

The deposit is represented as JSON input to the contract. The `marlowe-cli input deposit` tool conveniently formats the correct JSON for a deposit.

In [31]:
marlowe-cli input deposit --help

Usage: marlowe-cli input deposit --deposit-account PARTY --deposit-party PARTY 
                                 [--deposit-token TOKEN]
                                 --deposit-amount INTEGER 
                                 [--out-file OUTPUT_FILE]

  Create Marlowe input for a deposit.

Available options:
  --deposit-account PARTY  The account for the deposit.
  --deposit-party PARTY    The party making the deposit.
  --deposit-token TOKEN    The token being deposited, if not Ada.
  --deposit-amount INTEGER The amount of token being deposited.
  --out-file OUTPUT_FILE   JSON output file for contract input.
  -h,--help                Show this help text


In [32]:
marlowe-cli input deposit \
  --deposit-party Buyer \
  --deposit-account Seller \
  --deposit-amount "$PRICE" \
  --out-file input-2.json
json2yaml input-2.json

input_from_party:
  role_token: Buyer
into_account:
  role_token: Seller
of_token:
  currency_symbol: ''
  token_name: ''
that_deposits: 75000000


In [33]:
yaml2json << EOI > request-2.json
version: v1
inputs: [$(cat input-2.json)]
metadata: {}
tags: {}
EOI
cat request-2.json

{"inputs":[{"input_from_party":{"role_token":"Buyer"},"into_account":{"role_token":"Seller"},"of_token":{"currency_symbol":"","token_name":""},"that_deposits":75000000}],"metadata":{},"tags":{},"version":"v1"}


Next we post the request and store the response.

In [34]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $BUYER_ADDR" \
  -d @request-2.json \
  -o response-2.json \
  -sS
json2yaml response-2.json

links:
  transaction: contracts/3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20%231/transactions/a3e9a21f95ebf3df23e2e5c3de14dc02b8a87cf4578aa6afefec92a7e526e0e8
resource:
  contractId: 3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20#1
  transactionId: a3e9a21f95ebf3df23e2e5c3de14dc02b8a87cf4578aa6afefec92a7e526e0e8
  txBody:
    cborHex: 86aa00838258203376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20018258203376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a2002825820b32eab58a670cc0164c9c2c615c81b40dc9229ab8d899db8f232bbede0984107030d81825820b32eab58a670cc0164c9c2c615c81b40dc9229ab8d899db8f232bbede09841070312818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010183a200581d604959d439bd61c63c94a102be0defd6a54b36071d6789f05b7b0224a1011a3716a88ea300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a0496ed4002820058200655c39786c393d64a5eacdbf20396a43c166a4882ae57d1ac02daa450e2f620a200581d604959d

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [35]:
jq '.resource.txBody' response-2.json > tx-2.unsigned

In [36]:
TX_2=$(
marlowe-cli transaction submit \
  --tx-body-file tx-2.unsigned \
  --required-signer "$BUYER_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_2 = $TX_2"

TX_2 = a3e9a21f95ebf3df23e2e5c3de14dc02b8a87cf4578aa6afefec92a7e526e0e8


One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [37]:
echo "$EXPLORER_URL/transaction/$TX_2?tab=utxo"

https://preprod.cardanoscan.io/transaction/a3e9a21f95ebf3df23e2e5c3de14dc02b8a87cf4578aa6afefec92a7e526e0e8?tab=utxo


One can see that the buyer has approximately 75 ada less than originally.

In [38]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BUYER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
a3e9a21f95ebf3df23e2e5c3de14dc02b8a87cf4578aa6afefec92a7e526e0e8     0        924231822 lovelace + TxOutDatumNone
a3e9a21f95ebf3df23e2e5c3de14dc02b8a87cf4578aa6afefec92a7e526e0e8     2        1030090 lovelace + 1 bf35f1fb673d65bd7fc65e90e52213e7c70ceadfab633e9765be060b.4275796572 + TxOutDatumNone


The Marlowe contract still has the 2 ada from its creation and an additional 75 ada.

In [39]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_2#1"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
a3e9a21f95ebf3df23e2e5c3de14dc02b8a87cf4578aa6afefec92a7e526e0e8     1        77000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "0655c39786c393d64a5eacdbf20396a43c166a4882ae57d1ac02daa450e2f620"


## View the further progress of the contract on the blockchain

Marlowe Runtime\'s `HTTP` `GET` endpoint `/contracts/{contractId}/transactions/{transactionId}` can fetch a contract from the blockchain and return information about it.

In [40]:
curl -sS "$CONTRACT_URL/transactions/$TX_2" | json2yaml

links: {}
resource:
  block:
    blockHeaderHash: 139a50cd367cf8e04c1cadf4729cf9d3c8b03ae8456a33ab2198eceac72ea4d2
    blockNo: 732033
    slotNo: 23317666
  consumingTx: null
  continuations: null
  contractId: 3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20#1
  inputUtxo: 3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20#1
  inputs:
  - input_from_party:
      role_token: Buyer
    into_account:
      role_token: Seller
    of_token:
      currency_symbol: ''
      token_name: ''
    that_deposits: 75000000
  invalidBefore: 2023-03-16T21:07:33Z
  invalidHereafter: 2023-03-16T22:06:24Z
  metadata: {}
  outputContract:
    timeout: 1679007984000
    timeout_continuation: close
    when:
    - case:
        choose_between:
        - from: 0
          to: 0
        for_choice:
          choice_name: Everything is alright
          choice_owner:
            role_token: Buyer
      then: close
    - case:
        choose_between:
        - from: 1
          t

## Transaction 3: The Buyer Reports That There is a Problem

The buyer chooses to report a problem with the merchandise.

The choice is represented as JSON input to the contract. The `marlowe-cli input choose` tool conveniently formats the correct JSON for a choice.

In [41]:
marlowe-cli input choose --help

Usage: marlowe-cli input choose --choice-name NAME --choice-party PARTY
                                --choice-number INTEGER [--out-file OUTPUT_FILE]

  Create Marlowe input for a choice.

Available options:
  --choice-name NAME       The name of the choice made.
  --choice-party PARTY     The party making the choice.
  --choice-number INTEGER  The number chosen.
  --out-file OUTPUT_FILE   JSON output file for contract input.
  -h,--help                Show this help text


In [42]:
marlowe-cli input choose \
  --choice-name "Report problem" \
  --choice-party Buyer \
  --choice-number 1 \
  --out-file input-3.json
json2yaml input-3.json

for_choice_id:
  choice_name: Report problem
  choice_owner:
    role_token: Buyer
input_that_chooses_num: 1


In [43]:
yaml2json << EOI > request-3.json
version: v1
inputs: [$(cat input-3.json)]
metadata: {}
tags: {}
EOI
cat request-3.json

{"inputs":[{"for_choice_id":{"choice_name":"Report problem","choice_owner":{"role_token":"Buyer"}},"input_that_chooses_num":1}],"metadata":{},"tags":{},"version":"v1"}


Next we post the request and store the response.

In [44]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $BUYER_ADDR" \
  -d @request-3.json \
  -o response-3.json \
  -sS
json2yaml response-3.json

links:
  transaction: contracts/3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20%231/transactions/5a448f6b4861e36a984e880c6e46a7dacd8513de0fa27daf46f9c1d0ccac50af
resource:
  contractId: 3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20#1
  transactionId: 5a448f6b4861e36a984e880c6e46a7dacd8513de0fa27daf46f9c1d0ccac50af
  txBody:
    cborHex: 86aa0083825820a3e9a21f95ebf3df23e2e5c3de14dc02b8a87cf4578aa6afefec92a7e526e0e800825820a3e9a21f95ebf3df23e2e5c3de14dc02b8a87cf4578aa6afefec92a7e526e0e801825820a3e9a21f95ebf3df23e2e5c3de14dc02b8a87cf4578aa6afefec92a7e526e0e8020d81825820a3e9a21f95ebf3df23e2e5c3de14dc02b8a87cf4578aa6afefec92a7e526e0e80012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010183a200581d604959d439bd61c63c94a102be0defd6a54b36071d6789f05b7b0224a1011a370a9baea300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a0496ed400282005820c384ef66cdc894120d0f71b03aaa114fe3731b70f0c7155d9352a04a0aaec264a200581d604959d

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [45]:
jq '.resource.txBody' response-3.json > tx-3.unsigned

In [46]:
TX_3=$(
marlowe-cli transaction submit \
  --tx-body-file tx-3.unsigned \
  --required-signer "$BUYER_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_3 = $TX_3"

TX_3 = 5a448f6b4861e36a984e880c6e46a7dacd8513de0fa27daf46f9c1d0ccac50af


One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [47]:
echo "$EXPLORER_URL/transaction/$TX_3?tab=utxo"

https://preprod.cardanoscan.io/transaction/5a448f6b4861e36a984e880c6e46a7dacd8513de0fa27daf46f9c1d0ccac50af?tab=utxo


One can see that the buyer still has approximately 75 ada less than originally.

In [48]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BUYER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
5a448f6b4861e36a984e880c6e46a7dacd8513de0fa27daf46f9c1d0ccac50af     0        923442094 lovelace + TxOutDatumNone
5a448f6b4861e36a984e880c6e46a7dacd8513de0fa27daf46f9c1d0ccac50af     2        1030090 lovelace + 1 bf35f1fb673d65bd7fc65e90e52213e7c70ceadfab633e9765be060b.4275796572 + TxOutDatumNone


The Marlowe contract still has the 77 ada.

In [49]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_3#1"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
5a448f6b4861e36a984e880c6e46a7dacd8513de0fa27daf46f9c1d0ccac50af     1        77000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "c384ef66cdc894120d0f71b03aaa114fe3731b70f0c7155d9352a04a0aaec264"


## View the further progress of the contract on the blockchain

Marlowe Runtime\'s `HTTP` `GET` endpoint `/contracts/{contractId}/transactions/{transactionId}` can fetch a contract from the blockchain and return information about it.

In [50]:
curl -sS "$CONTRACT_URL/transactions/$TX_3" | json2yaml

links:
  previous: contracts/3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20%231/transactions/a3e9a21f95ebf3df23e2e5c3de14dc02b8a87cf4578aa6afefec92a7e526e0e8
resource:
  block:
    blockHeaderHash: c58f330e1c27404c9643dd27694dd027301191b4c8fa8eca0960db85f1a70615
    blockNo: 732035
    slotNo: 23317702
  consumingTx: null
  continuations: null
  contractId: 3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20#1
  inputUtxo: a3e9a21f95ebf3df23e2e5c3de14dc02b8a87cf4578aa6afefec92a7e526e0e8#1
  inputs:
  - for_choice_id:
      choice_name: Report problem
      choice_owner:
        role_token: Buyer
    input_that_chooses_num: 1
  invalidBefore: 2023-03-16T21:07:52Z
  invalidHereafter: 2023-03-16T23:06:24Z
  metadata: {}
  outputContract:
    timeout: 1679011584000
    timeout_continuation: close
    when:
    - case:
        choose_between:
        - from: 1
          to: 1
        for_choice:
          choice_name: Confirm problem
          choice_owner:
  

## Transaction 4: The Seller Disputes that There is a Problem

Now the seller chooses to dispute that there is a problem with the merchandise.

In [51]:
marlowe-cli input choose \
  --choice-name "Dispute problem" \
  --choice-party Seller \
  --choice-number 0 \
  --out-file input-4.json
json2yaml input-4.json

for_choice_id:
  choice_name: Dispute problem
  choice_owner:
    role_token: Seller
input_that_chooses_num: 0


In [52]:
yaml2json << EOI > request-4.json
version: v1
inputs: [$(cat input-4.json)]
metadata: {}
tags: {}
EOI
cat request-4.json

{"inputs":[{"for_choice_id":{"choice_name":"Dispute problem","choice_owner":{"role_token":"Seller"}},"input_that_chooses_num":0}],"metadata":{},"tags":{},"version":"v1"}


Next we post the request and store the response.

In [53]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $SELLER_ADDR" \
  -d @request-4.json \
  -o response-4.json \
  -sS
json2yaml response-4.json

links:
  transaction: contracts/3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20%231/transactions/0f40ff5e41521e03a15837f3f1ee1436103ed64a1b55c5698bdd8a8db6ab043c
resource:
  contractId: 3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20#1
  transactionId: 0f40ff5e41521e03a15837f3f1ee1436103ed64a1b55c5698bdd8a8db6ab043c
  txBody:
    cborHex: 86aa00838258203376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20048258205a448f6b4861e36a984e880c6e46a7dacd8513de0fa27daf46f9c1d0ccac50af01825820b32eab58a670cc0164c9c2c615c81b40dc9229ab8d899db8f232bbede0984107020d81825820b32eab58a670cc0164c9c2c615c81b40dc9229ab8d899db8f232bbede09841070212818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010183a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc59011a3b9025c2a300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a0496ed4002820058203431d4dd22e93ac8a724511fd960b137d98905a2aad7b59ee299790f9c9f7f2ca200581d601b120

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [54]:
jq '.resource.txBody' response-4.json > tx-4.unsigned

In [55]:
TX_4=$(
marlowe-cli transaction submit \
  --tx-body-file tx-4.unsigned \
  --required-signer "$SELLER_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_4 = $TX_4"

TX_4 = 0f40ff5e41521e03a15837f3f1ee1436103ed64a1b55c5698bdd8a8db6ab043c


One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [56]:
echo "$EXPLORER_URL/transaction/$TX_4?tab=utxo"

https://preprod.cardanoscan.io/transaction/0f40ff5e41521e03a15837f3f1ee1436103ed64a1b55c5698bdd8a8db6ab043c?tab=utxo


One can see that the seller still has approximately their original balance.

In [57]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$SELLER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
0f40ff5e41521e03a15837f3f1ee1436103ed64a1b55c5698bdd8a8db6ab043c     0        999302594 lovelace + TxOutDatumNone
0f40ff5e41521e03a15837f3f1ee1436103ed64a1b55c5698bdd8a8db6ab043c     2        1034400 lovelace + 1 bf35f1fb673d65bd7fc65e90e52213e7c70ceadfab633e9765be060b.53656c6c6572 + TxOutDatumNone


The Marlowe contract still has the 77 ada.

In [58]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_4#1"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
0f40ff5e41521e03a15837f3f1ee1436103ed64a1b55c5698bdd8a8db6ab043c     1        77000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "3431d4dd22e93ac8a724511fd960b137d98905a2aad7b59ee299790f9c9f7f2c"


## View the further progress of the contract on the blockchain

Marlowe Runtime\'s `HTTP` `GET` endpoint `/contracts/{contractId}/transactions/{transactionId}` can fetch a contract from the blockchain and return information about it.

In [59]:
curl -sS "$CONTRACT_URL/transactions/$TX_4" | json2yaml

links:
  previous: contracts/3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20%231/transactions/5a448f6b4861e36a984e880c6e46a7dacd8513de0fa27daf46f9c1d0ccac50af
resource:
  block:
    blockHeaderHash: d46d1d6a3554c29d439fd818cec68a8deb0c46f31efdc17c7161a2287fe15103
    blockNo: 732037
    slotNo: 23317731
  consumingTx: null
  continuations: null
  contractId: 3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20#1
  inputUtxo: 5a448f6b4861e36a984e880c6e46a7dacd8513de0fa27daf46f9c1d0ccac50af#1
  inputs:
  - for_choice_id:
      choice_name: Dispute problem
      choice_owner:
        role_token: Seller
    input_that_chooses_num: 0
  invalidBefore: 2023-03-16T21:08:25Z
  invalidHereafter: 2023-03-17T00:06:24Z
  metadata: {}
  outputContract:
    timeout: 1679015184000
    timeout_continuation: close
    when:
    - case:
        choose_between:
        - from: 0
          to: 0
        for_choice:
          choice_name: Dismiss claim
          choice_owner:
  

## Transaction 5: The Mediator Dismisses the Claim

The mediator rules in favor or the seller and dismisses the buyer's claim.

In [60]:
marlowe-cli input choose \
  --choice-name "Dismiss claim" \
  --choice-party Mediator \
  --choice-number 0 \
  --out-file input-5.json
json2yaml input-5.json

for_choice_id:
  choice_name: Dismiss claim
  choice_owner:
    role_token: Mediator
input_that_chooses_num: 0


In [61]:
yaml2json << EOI > request-5.json
version: v1
inputs: [$(cat input-5.json)]
metadata: {}
tags: {}
EOI
cat request-5.json

{"inputs":[{"for_choice_id":{"choice_name":"Dismiss claim","choice_owner":{"role_token":"Mediator"}},"input_that_chooses_num":0}],"metadata":{},"tags":{},"version":"v1"}


Next we post the request and store the response.

In [62]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $MEDIATOR_ADDR" \
  -d @request-5.json \
  -o response-5.json \
  -sS
json2yaml response-5.json

links:
  transaction: contracts/3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20%231/transactions/182a53706d9eedb1b66d8e86d54351ba8a7822fc2361e1cdb6a129f265d22040
resource:
  contractId: 3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20#1
  transactionId: 182a53706d9eedb1b66d8e86d54351ba8a7822fc2361e1cdb6a129f265d22040
  txBody:
    cborHex: 86aa00838258200f40ff5e41521e03a15837f3f1ee1436103ed64a1b55c5698bdd8a8db6ab043c018258203376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20008258203376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20030d818258203376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a200012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010184a200581d60f4b22c10898e0465d771cbb89862a1d715838aba619c39fc7ebe98dd011a3b39ea31a200581d60f4b22c10898e0465d771cbb89862a1d715838aba619c39fc7ebe98dd01821a000fea4ca1581cbf35f1fb673d65bd7fc65e90e52213e7c70ceadfab633e9765be060ba1484d65646961746f7201a30

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [63]:
jq '.resource.txBody' response-5.json > tx-5.unsigned

In [64]:
TX_5=$(
marlowe-cli transaction submit \
  --tx-body-file tx-5.unsigned \
  --required-signer "$MEDIATOR_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_5 = $TX_5"

TX_5 = 182a53706d9eedb1b66d8e86d54351ba8a7822fc2361e1cdb6a129f265d22040


One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [65]:
echo "$EXPLORER_URL/transaction/$TX_5?tab=utxo"

https://preprod.cardanoscan.io/transaction/182a53706d9eedb1b66d8e86d54351ba8a7822fc2361e1cdb6a129f265d22040?tab=utxo


One can see that the mediator still has approximately their original balance, which includes the 2 ada they just received, refunding the 2 ada they used to create the contract.

In [66]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$MEDIATOR_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
182a53706d9eedb1b66d8e86d54351ba8a7822fc2361e1cdb6a129f265d22040     0        993651249 lovelace + TxOutDatumNone
182a53706d9eedb1b66d8e86d54351ba8a7822fc2361e1cdb6a129f265d22040     1        1043020 lovelace + 1 bf35f1fb673d65bd7fc65e90e52213e7c70ceadfab633e9765be060b.4d65646961746f72 + TxOutDatumNone
182a53706d9eedb1b66d8e86d54351ba8a7822fc2361e1cdb6a129f265d22040     3        2000000 lovelace + TxOutDatumNone


The Marlowe contract is closed, but the role-payout address has the 75 ada for the benefit of the seller.

In [67]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_5#2"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
182a53706d9eedb1b66d8e86d54351ba8a7822fc2361e1cdb6a129f265d22040     2        75000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "1fc22e57cbb141ffcba4d63f002e03cb55031186880e32e5448e526044eaacd5"


## View the further progress of the contract on the blockchain

Marlowe Runtime\'s `HTTP` `GET` endpoint `/contracts/{contractId}/transactions/{transactionId}` can fetch a contract from the blockchain and return information about it.

In [68]:
curl -sS "$CONTRACT_URL/transactions/$TX_5" | json2yaml

links:
  previous: contracts/3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20%231/transactions/0f40ff5e41521e03a15837f3f1ee1436103ed64a1b55c5698bdd8a8db6ab043c
resource:
  block:
    blockHeaderHash: 23a80ec90982f747cba7f17992e673572a3d4b06d94bbfa13e1a62f3c9054396
    blockNo: 732038
    slotNo: 23317766
  consumingTx: null
  continuations: null
  contractId: 3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20#1
  inputUtxo: 0f40ff5e41521e03a15837f3f1ee1436103ed64a1b55c5698bdd8a8db6ab043c#1
  inputs:
  - for_choice_id:
      choice_name: Dismiss claim
      choice_owner:
        role_token: Mediator
    input_that_chooses_num: 0
  invalidBefore: 2023-03-16T21:08:51Z
  invalidHereafter: 2023-03-17T01:06:24Z
  metadata: {}
  outputContract: null
  outputState: null
  outputUtxo: null
  status: confirmed
  tags: {}
  transactionId: 182a53706d9eedb1b66d8e86d54351ba8a7822fc2361e1cdb6a129f265d22040
  txBody: null


## Transaction 6: The Seller Withdraws Their Funds

The price of 75 ada is held at Marlowe's role-payout address for the benefit of the seller. The seller can withdraw these funds at any time. The contract ID and role name are included in the request body for a withdrawal.

In [69]:
yaml2json << EOI > request-6.json
contractId: "$CONTRACT_ID"
role: Seller
EOI
cat request-6.json

{"contractId":"3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20#1","role":"Seller"}


Next we post the request and store the response.

In [70]:
curl "$MARLOWE_RT_WEBSERVER_URL/withdrawals" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $SELLER_ADDR" \
  -d @request-6.json \
  -o response-6.json \
  -sS
json2yaml response-6.json

links:
  withdrawal: withdrawals/850e4ce20d1c1b07f05c28d7d6c9d961597d1c29b90d6cf57f35f4629461ab5a
resource:
  txBody:
    cborHex: 86a800838258200f40ff5e41521e03a15837f3f1ee1436103ed64a1b55c5698bdd8a8db6ab043c008258200f40ff5e41521e03a15837f3f1ee1436103ed64a1b55c5698bdd8a8db6ab043c02825820182a53706d9eedb1b66d8e86d54351ba8a7822fc2361e1cdb6a129f265d22040020d818258200f40ff5e41521e03a15837f3f1ee1436103ed64a1b55c5698bdd8a8db6ab043c0012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd020182a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc59011a40037f60a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc5901821a000fc8a0a1581cbf35f1fb673d65bd7fc65e90e52213e7c70ceadfab633e9765be060ba14653656c6c65720110a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc59011a3b888f0f111a000796b3021a00050f220b5820eb4a2171f8674f1256d443350ae57c518900b0398bdd259564e6935869d5c35e9fff81d8799f581cbf35f1fb673d65bd7fc65e90e52213e7c70ceadfab633e9765be060b46536

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [71]:
jq '.resource.txBody' response-6.json > tx-6.unsigned

In [72]:
TX_6=$(
marlowe-cli transaction submit \
  --tx-body-file tx-6.unsigned \
  --required-signer "$SELLER_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_6 = $TX_6"

TX_6 = 850e4ce20d1c1b07f05c28d7d6c9d961597d1c29b90d6cf57f35f4629461ab5a


On can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [73]:
echo "$EXPLORER_URL/transaction/$TX_6?tab=utxo"

https://preprod.cardanoscan.io/transaction/850e4ce20d1c1b07f05c28d7d6c9d961597d1c29b90d6cf57f35f4629461ab5a?tab=utxo


The seller now has about an additional 75 ada.

In [74]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$SELLER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
850e4ce20d1c1b07f05c28d7d6c9d961597d1c29b90d6cf57f35f4629461ab5a     0        1073971040 lovelace + TxOutDatumNone
850e4ce20d1c1b07f05c28d7d6c9d961597d1c29b90d6cf57f35f4629461ab5a     1        1034400 lovelace + 1 bf35f1fb673d65bd7fc65e90e52213e7c70ceadfab633e9765be060b.53656c6c6572 + TxOutDatumNone


## View the withdrawal

Marlowe Runtime\'s `HTTP` `GET` endpoint `/withdrawals/{transactionId}` can fetch a withdrawal from the blockchain and return information about it.

In [75]:
curl -sS "$MARLOWE_RT_WEBSERVER_URL/withdrawals/$TX_6" | json2yaml

block:
  blockHeaderHash: 4dfa1ea56ce98d4f8a476e808ce5f5c2803ffd76d4b8ba3ee83f57ec816d9f66
  blockNo: 732039
  slotNo: 23317779
payouts:
- contractId: 3376c0eae5aa40e75e5d1d2264faa1a30a314a3d8ca9b6da99c5c04de9349a20#1
  payout: 182a53706d9eedb1b66d8e86d54351ba8a7822fc2361e1cdb6a129f265d22040#2
  role: Seller
  roleTokenMintingPolicyId: bf35f1fb673d65bd7fc65e90e52213e7c70ceadfab633e9765be060b
status: confirmed
withdrawalId: 850e4ce20d1c1b07f05c28d7d6c9d961597d1c29b90d6cf57f35f4629461ab5a
